# Bayesian binomial AB test calculator

Instructions: Press the Run button above to set the notebook up. There will be several interactive elements (noted as "interactive") where you can input parameters and get output.

In [1]:
import numpy as np
from scipy.stats import beta as betadist
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets

%matplotlib inline

## Get $\alpha$ and $\beta$ parameters for Beta distribution

Uses method of moments

In [2]:
def compute_alpha_beta(mu, sigma):
    """Method of moments calculation of Beta distribution parameters"""
    try:
        mu = float(mu)
        sigma = float(sigma)
    except:
        print("Need to give valid floating point numbers")
        return
    if not 0 < mu < 1:
        print("Need to give mu in a the range (0, 1)")
        return
    if not 0 < sigma < 1:
        print("Need to give sigma in a the range (0, 1)")
        return
    alpha = -mu + ((1 - mu)*mu**2)/(sigma**2)
    beta = -alpha + (alpha/mu)
    print("alpha: %.2f, beta: %.2f" % (alpha, beta))

### Interactive

In [3]:
interact(compute_alpha_beta, mu='0.2', sigma='0.03');

interactive(children=(Text(value='0.2', description='mu'), Text(value='0.03', description='sigma'), Output()),…

## Plot Beta distributions - prior and posterior

In [4]:
def plot_betas(alpha, beta, successes, total_samples):
    alpha = float(alpha)
    beta = float(beta)
    successes = float(successes)
    total_samples = float(total_samples)
    
    x = np.linspace(0, 1.0, 1000)
    y1 = betadist.pdf(x, alpha, beta)
    y2 = betadist.pdf(x, alpha + successes, beta + (total_samples - successes))
    
    plt.figure(figsize=(4,4), dpi=300)
    plt.plot(x, y1, linewidth=2, color='k', linestyle=':', label='Prior')
    plt.plot(x, y2, linewidth=2, color='r', label='Posterior')

    plt.legend(fontsize=16)
    
    plt.show()

### Interactive

In [5]:
interact(
    plot_betas,
    alpha='35.36',
    beta='141.42',
    successes='45',
    total_samples='200');

interactive(children=(Text(value='35.36', description='alpha'), Text(value='141.42', description='beta'), Text…

## Chance option A is better than option B

In [6]:
def chance_option_A_true(s_a, n_a, s_b, n_b, alpha, beta, x_steps=5000):
    """
    Inputs:
    - s_a: Number of succeses for option A
    - n_a: Number of total samples for option A
    - s_b: Number of successes for option B
    - n_b: Number of total samples for option B
    - alpha: Alpha
    - beta: Beta
    - x_steps: Number of steps for numerical integral
    """
    s_a = float(s_a)
    s_b = float(s_b)
    n_a = float(n_a)
    n_b = float(n_b)
    a = float(alpha)
    b = float(beta)
    
    f_a = n_a - s_a
    f_b = n_b - s_b
    x = np.linspace(0, 1, num=x_steps, endpoint=False)
    dx = x[1] - x[0]
    beta_a = betadist.pdf(x, a + s_a, b + f_a)
    beta_b = betadist.pdf(x, a + s_b, b + f_b)
    
    array = np.outer(beta_a, beta_b)
    array = np.tril(array)  # Mask out upper-triangular bit
    total = array.sum().sum()*dx**2

    print("Chance A >= B %.2f%%" % (100*total))

### Interactive

In [7]:
interact(
    chance_option_A_true,
    s_a='45',
    n_a='200',
    s_b='40',
    n_b='200',
    alpha='35.36',
    beta='141.42',
    x_steps=5000);

interactive(children=(Text(value='45', description='s_a'), Text(value='200', description='n_a'), Text(value='4…

## Expected risk when choosing option A

In [8]:
def loss_option_A(s_a, n_a, s_b, n_b, alpha, beta, x_steps=5000):
    """
    Inputs:
    - s_a: Number of succeses for option A
    - n_a: Number of total samples for option A
    - s_b: Number of successes for option B
    - n_b: Number of total samples for option B
    - alpha: Alpha
    - beta: Beta
    - x_steps: Number of steps for numerical integral
    """
    s_a = float(s_a)
    s_b = float(s_b)
    n_a = float(n_a)
    n_b = float(n_b)
    a = float(alpha)
    b = float(beta)

    f_a = n_a - s_a
    f_b = n_b - s_b
    x = np.linspace(0, 1, num=x_steps, endpoint=False)
    dx = x[1] - x[0]
    beta_a = betadist.pdf(x, a + s_a, b + f_a)
    beta_b = betadist.pdf(x, a + s_b, b + f_b)
    
    joint_posterior = np.outer(beta_a, beta_b)
    loss_error = np.fromfunction(lambda i, j: (j - i)*dx, (x_steps, x_steps))

    loss_error = np.triu(loss_error)  # Mask out lower-triangular bit
    final = joint_posterior * loss_error
    
    total = final.sum().sum()*dx**2

    if total*100 >= 0.01:
        print("Expected loss with option A: %.2f%%" % (100*total))
    else:
        print("Expected loss with option A: %.2e%%" % (100*total))

### Interactive

In [9]:
interact(
    loss_option_A,
    s_a='45',
    n_a='200',
    s_b='40',
    n_b='200',
    alpha='35.36',
    beta='141.42',
    x_steps=5000);

interactive(children=(Text(value='45', description='s_a'), Text(value='200', description='n_a'), Text(value='4…